In [8]:
import urllib2
from bs4 import BeautifulSoup as Soup
from lxml import etree
import time
import pandas as pd
import re

def get_all_subjects(url):
    res = []
    res += get_subjects(url)
    while has_next_page(url):
        new_url = url[0:len(url)-1] + str(int(url[len(url)-1])+1)
        res += get_subjects(new_url)
        url = new_url
    return res
        
def has_next_page(url):
    page = urllib2.urlopen(url)
    soup = Soup(page)
    if soup.find_all('a', {"href": url[0:len(url)-1] + str(int(url[len(url)-1])+1)}):
        return True
    else:
        return False

def get_subjects(url):
    page = urllib2.urlopen(url)
    soup = Soup(page)
    sections = soup.find_all('ul', {"class": "plain margin7"})
    res = []
    for section in sections:
        for listelement in section.findAll('li'):
            res.append(listelement.get_text())
    return res
    
def find_bill_subjects(billName):
    if billName[0:2] == "PN":
        return []
    else:
        bill = billName.split()
        if bill[0] == 'H.R.':
            url = "https://www.congress.gov/bill/114th-congress/house-bill/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
        elif bill[0] == 'S.':
            url = "https://www.congress.gov/bill/114th-congress/senate-bill/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
        elif bill[0] == 'S.J.Res.':
            url = "https://www.congress.gov/bill/114th-congress/senate-joint-resolution/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
        elif bill[0] == 'H.J.Res.':
            url = "https://www.congress.gov/bill/114th-congress/house-joint-resolution/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
        elif bill[0] == 'S.Con.Res.':
            url = "https://www.congress.gov/bill/114th-congress/senate-concurrent-resolution/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
            
print find_bill_subjects("H.R. 26")

[u'Administrative law and regulatory procedures', u'Advisory bodies', u'Banking and financial institutions regulation', u'Business records', u'Civil actions and liability', u'Commodities markets', u'Congressional oversight', u'Consumer affairs', u'Criminal justice information and records', u'Department of the Treasury', u'Disaster relief and insurance', u'Federal Reserve System', u'Federal officials', u'Federal preemption', u'Finance and Financial Sector', u'Financial services and investments', u'Government corporations and government-sponsored enterprises', u'Government studies and investigations', u'Insurance industry and regulation', u'Licensing and registrations', u'Life, casualty, property insurance', u'Performance measurement', u'Securities', u'Small business', u'State and local government operations', u'Terrorism']


In [9]:
def get_bill_sponsor(url):
    page = urllib2.urlopen(url)
    soup = Soup(page)
    sponsor_dirty =  soup.find('th', text = re.compile('Sponsor:'), attrs = {"scope": "row"}).findNext('td').find('a').text.split()
    sponsor_clean = sponsor_dirty[1][:-1] + " (" + sponsor_dirty[len(sponsor_dirty)-1][1:5] + ")"
    return sponsor_clean

def find_bill_sponsor(billTitle):
    if billTitle[0:2] == "PN":
        return "No sponsor"
    else:
        bill = billTitle.split()
        if bill[0] == 'H.R.':
            url = "https://www.congress.gov/bill/114th-congress/house-bill/" + bill[1]
            return get_bill_sponsor(url)
        elif bill[0] == 'S.':
            url = "https://www.congress.gov/bill/114th-congress/senate-bill/" + bill[1]
            return get_bill_sponsor(url)
        elif bill[0] == 'S.J.Res.':
            url = "https://www.congress.gov/bill/114th-congress/senate-joint-resolution/" + bill[1]
            return get_bill_sponsor(url)
        elif bill[0] == 'H.J.Res.':
            url = "https://www.congress.gov/bill/114th-congress/house-joint-resolution/" + bill[1]
            return get_bill_sponsor(url)
        elif bill[0] == 'S.Con.Res.':
            url = "https://www.congress.gov/bill/114th-congress/senate-concurrent-resolution/" + bill[1]
            return get_bill_sponsor(url)
        elif bill[0] == 'S.Amdt.':
            url = "https://www.congress.gov/amendment/114th-congress/senate-amendment/" + bill[1]
            return get_bill_sponsor(url)

print find_bill_sponsor('S.Amdt. 1')

Warren (D-MA)


In [10]:
#339 is the max number for this session
tree = etree.parse('http://www.senate.gov/legislative/LIS/roll_call_votes/vote1141/vote_114_1_00001.xml')

vote_date = tree.findall("vote_date")[0].text
vote_result = tree.findall("vote_result")[0].text
vote_title = tree.findall("document")[0][3].text
if vote_title:
    subjects = find_bill_subjects(vote_title)
    bill_title = vote_title
    sponsor = find_bill_sponsor(bill_title)
elif not vote_title:
    vote_title = "Amendment to " + tree.findall("amendment")[0][3].text
    subjects = find_bill_subjects(tree.findall("amendment")[0][3].text)
    bill_title = tree.findall("amendment")[0][0].text
    sponsor = find_bill_sponsor(bill_title)
    if not vote_title:
        subjects = find_bill_subjects("PN")
        sponsor = find_bill_sponsor("PN")
        bill_title = "Bro I have no idea what this about"
        vote_title = "Bro I have no idea what this about"

vote_dict = {'voteDate': [vote_date], 'title': [vote_title], 'voteResult': [vote_result], 'subjects': [subjects], 
             'billTitle': [bill_title], 'sponsor': [sponsor]}
for senator in tree.getiterator("member"):
    vote_dict[senator[0].text] = [senator[5].text]
    
#initializes dictionary

for i in range(2, 340): #339 is the max number of bills in this session of Congress
    if i % 10 == 0:
        print i
    url = "http://www.senate.gov/legislative/LIS/roll_call_votes/vote1141/vote_114_1_" + '{0:05d}'.format(i) + '.xml'
    tree = etree.parse(url)
    vote_date = tree.findall("vote_date")[0].text
    vote_result = tree.findall("vote_result")[0].text
    vote_title = tree.findall("document")[0][3].text
    if vote_title:
        bill_title = vote_title
        subjects = find_bill_subjects(vote_title)
        sponsor = find_bill_sponsor(bill_title)
    elif not vote_title:
        vote_title = "Amendment to " + tree.findall("amendment")[0][3].text
        subjects = find_bill_subjects(tree.findall("amendment")[0][3].text)
        bill_title = tree.findall("amendment")[0][0].text
        sponsor = find_bill_sponsor(bill_title)
        if not vote_title:
            subjects = find_bill_subjects("PN")
            sponsor = find_bill_sponsor("PN")
            bill_title = "Bro I have no idea what this about"
            vote_title = "Bro I have no idea what this about"
    vote_dict['voteDate'].append(vote_date)
    vote_dict['title'].append(vote_title)
    vote_dict['voteResult'].append(vote_result)
    vote_dict['subjects'].append(subjects)
    vote_dict['billTitle'].append(bill_title)
    vote_dict['sponsor'].append(sponsor)
    for senator in tree.getiterator("member"):
        vote_dict[senator[0].text].append(senator[5].text)
    time.sleep(1)
    
print len(vote_dict['sponsor'])

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
339


In [11]:
df = pd.DataFrame(data=vote_dict)
df.to_csv('rollCallVotes_iter4.csv', index=False)